# Bot Trader de Bitcoin
Este notebook implementa un bot de trading algorítmico para Bitcoin (BTC) con lógica mejorada, indicadores técnicos y gestión básica de riesgo.

## 1. Importar librerías necesarias
Importamos las librerías requeridas para la obtención de datos, análisis, visualización y cálculo de indicadores técnicos.

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go

## 2. Obtención de datos históricos de BTC
Descargamos los datos históricos de Bitcoin (BTC) en USD usando yfinance, con intervalos de 5 minutos para los últimos 7 días.

In [2]:
def importar_base_bitcoin():
    global df_bitcoin
    datos = yf.download('BTC-USD', period='7d', interval='5m', auto_adjust=True)
    df_bitcoin = pd.DataFrame(datos)
    df_bitcoin.index.name = 'Datetime'

In [3]:
importar_base_bitcoin()
df_bitcoin.tail()

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Datetime,,,,,
2025-08-13 16:30:00+00:00,121572.382812,121581.351562,121534.429688,121581.351562,1089667072
2025-08-13 16:35:00+00:00,121593.429688,121698.804688,121593.429688,121625.453125,1213022208
2025-08-13 16:40:00+00:00,121350.984375,121563.375000,121350.984375,121563.375000,4669440
2025-08-13 16:45:00+00:00,121374.140625,121376.500000,121356.984375,121356.984375,3066773504
2025-08-13 16:50:00+00:00,121371.828125,121381.554688,121287.023438,121381.554688,25600000


## 3. Web Scraping: Precio y tendencia actual de BTC
Obtenemos el precio y la tendencia actual de BTC desde CoinMarketCap usando BeautifulSoup.

In [4]:
def extraer_tendencias():
    global precio_actual, tendencia
    url = 'https://coinmarketcap.com/'
    respuesta = requests.get(url)
    if respuesta.status_code == 200:
        s = BeautifulSoup(respuesta.content, 'lxml')
        html_row = s.find_all('tr')[1]
        elementos = html_row.find_all('span')
        precio_actual = float(elementos[1].text.replace('$', '').replace(',', ''))
        if elementos[2].next.attrs['class'][0] == 'icon-Caret-up':
            tendencia = 'Alta'
        else:
            tendencia = 'Baja'
    else:
        precio_actual, tendencia = np.nan, 'Desconocida'

In [5]:
extraer_tendencias()
print(f'Precio actual: ${precio_actual} | Tendencia: {tendencia}')

Precio actual: $121381.55 | Tendencia: Alta


## 4. Limpieza y simplificación de datos
Eliminamos columnas innecesarias, registros con volumen cero y duplicados.

In [6]:
def limpiar_datos():
    global df_resumen
    df_resumen = df_bitcoin[['Close', 'Volume']].copy()
    df_resumen.columns = pd.Index(['Close', 'Volume'])
    df_resumen.index.name = None
    df_resumen = df_resumen[df_resumen['Volume'] > 0]
    df_resumen = df_resumen.drop_duplicates()

In [7]:
limpiar_datos()
df_resumen.describe()

,Close,Volume
count,974.000000,9.740000e+02
mean,118261.431388,5.697989e+08
std,1834.242659,5.628576e+08
min,114381.437500,5.734400e+05
25%,116695.892578,8.741683e+07
50%,118390.117188,3.509064e+08
75%,119624.339844,1.040865e+09
max,122257.460938,3.976126e+09


## 5. Indicadores técnicos: Medias móviles y RSI
Calculamos la media móvil simple (SMA), la media móvil exponencial (EMA) y el RSI para la toma de decisiones.

In [8]:
def calcular_indicadores():
    global df_resumen
    df_resumen['SMA_50'] = df_resumen['Close'].rolling(window=50).mean()
    df_resumen['EMA_20'] = df_resumen['Close'].ewm(span=20, adjust=False).mean()
    delta = df_resumen['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df_resumen['RSI'] = 100 - (100 / (1 + rs))

In [9]:
calcular_indicadores()
df_resumen[['Close', 'SMA_50', 'EMA_20', 'RSI']].tail()

,Close,SMA_50,EMA_20,RSI
2025-08-13 16:30:00+00:00,121572.382812,120974.556094,121277.924735,72.909912
2025-08-13 16:35:00+00:00,121593.429688,120999.326094,121307.972826,75.343858
2025-08-13 16:40:00+00:00,121350.984375,121020.053281,121312.069164,67.009894
2025-08-13 16:45:00+00:00,121374.140625,121039.759063,121317.980731,60.748190
2025-08-13 16:50:00+00:00,121371.828125,121058.876562,121323.109055,51.680588


In [10]:
df_resumen.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 974 entries, 2025-08-07 00:20:00+00:00 to 2025-08-13 16:50:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   974 non-null    float64
 1   Volume  974 non-null    int64  
 2   SMA_50  925 non-null    float64
 3   EMA_20  974 non-null    float64
 4   RSI     961 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 45.7 KB


## 6. Estrategia de trading
La decisión de trading se basa en la relación entre el precio actual, las medias móviles y el RSI. Se añade un umbral para evitar señales falsas y se recomienda gestión de riesgo.

In [11]:
def tomar_decision_mejorada(umbral=0.002, rsi_compra=35, rsi_venta=65):
    global decision
    precio = df_resumen['Close'].iloc[-1]
    sma = df_resumen['SMA_50'].iloc[-1]
    ema = df_resumen['EMA_20'].iloc[-1]
    rsi = df_resumen['RSI'].iloc[-1]
    if (precio > sma * (1 + umbral)) and (precio > ema) and (rsi > rsi_venta):
        decision = 'Vender'
    elif (precio < sma * (1 - umbral)) and (precio < ema) and (rsi < rsi_compra):
        decision = 'Comprar'
    else:
        decision = 'Esperar'

In [12]:
tomar_decision_mejorada()
print(f'Decisión de trading: {decision}')

Decisión de trading: Esperar


## 7. Visualización de precios, indicadores y decisión
Mostramos el precio, las medias móviles, el RSI y la decisión tomada en un gráfico interactivo.

In [13]:
def grafico_btc_mejorado():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_resumen.index, y=df_resumen['Close'], name='Precio Bitcoin'))
    fig.add_trace(go.Scatter(x=df_resumen.index, y=df_resumen['SMA_50'], name='SMA 50'))
    fig.add_trace(go.Scatter(x=df_resumen.index, y=df_resumen['EMA_20'], name='EMA 20'))
    color = 'green' if decision == 'Vender' else 'blue' if decision == 'Comprar' else 'orange'
    fig.add_annotation(
        x=df_resumen.index[-1], 
        y=df_resumen['Close'].iloc[-1], 
        text=decision, 
        showarrow=True, 
        font=dict(size=12, color=color), 
        arrowhead=2, arrowsize=1, arrowwidth=2, arrowcolor=color, ax=0, ay=-50, bordercolor=color, borderwidth=2, borderpad=4, bgcolor='white', opacity=0.8
    )
    fig.update_layout(title='Precio de Bitcoin, medias móviles y decisión', xaxis_title='Fecha', yaxis_title='Precio (USD)')
    fig.show()

In [14]:
grafico_btc_mejorado()

## 8. Automatización del análisis
Puedes automatizar el análisis repitiendo el proceso cada cierto tiempo, usando un bucle y control de interrupciones.

In [ ]:
import time
from IPython.display import clear_output
try:
    while True:
        clear_output()
        importar_base_bitcoin()
        extraer_tendencias()
        limpiar_datos()
        calcular_indicadores()
        tomar_decision_mejorada()
        grafico_btc_mejorado()
        print(f'Precio actual: ${precio_actual} | Tendencia: {tendencia} | Decisión: {decision}')
        time.sleep(300)
except KeyboardInterrupt:
    print('Ejecución detenida por el usuario.')

[*********************100%***********************]  1 of 1 completed


Precio actual: $121381.55 | Tendencia: Alta | Decisión: Esperar
